In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import re
import time
import os
# for trend detection
import sys
sys.path.append("..")

from configs import bad_direction_kpi_dict as dir_dict
from configs import kpi_constraints_dict
from configs.tests2 import drop_missing_data, test_stationarity, test_seasonality, ACF_condition
from configs.tests2 import test_seasonality_vectorized, pivot_maker_vectorized, seas_testing_vectorized

from collections import OrderedDict
# kpi_constraints_dict for is_it_constant

from statsmodels.tsa.stattools import acf
from datetime import datetime
import datetime

#pd.set_option('display.max_rows', None)
import dask.dataframe as dd

In [2]:
# Data related stuff #
def regex_file_filter(s, patt='4weeks-\S*\.csv'):
    pattern = re.compile(patt)
    return pattern.match(s) != None


def clean(df, col_id):
    # dropping useless rows and columns
    df_clean = df.copy()
    df_clean = df_clean[df_clean[col_id].notnull()] # useful rows

    df_desc = df_clean.describe(datetime_is_numeric=True).T
    cols_to_use = list(df_desc[(df_desc['std']!=0.0)&(df_desc['count']>1.0)].index) # useful cols
    
    if (col_id in cols_to_use): return df_clean[cols_to_use]
    return df_clean[[col_id, *cols_to_use]]


def data_step(string_file, col_id, sep=',', index_col=0):
    df = pd.read_csv(string_file, sep=sep, index_col=index_col)
    df_clean = clean(df, col_id)

    df_clean['dt'] = pd.to_datetime(df_clean.ts, unit='s') # gonna be the index
    return df_clean.sort_values(by='ts')


trim = lambda ser: ser.loc[ser.first_valid_index():ser.last_valid_index()]


def rowIndex(row):
    return row.name


def colIndex(row, new_end):
    col_idx, = np.where(np.array(row.index) == pd.Timestamp(new_end))
    return col_idx[0]


In [3]:
file = "/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-lte_eci-1614153600.csv"
ts = "1611734400"
max_missing_ratio = .35
results = pd.DataFrame(columns=['ts',
                                'path',
                                'dimension_name',
                                'kpi_name',
                                'missing_data_ratio_all', 
                                'missing_data_ratio_last_week',
                                'seasonality_flag',
                                'statonarity_flag',
                                'missing_data_imputation_flag',
                                'table',
                                "nan_trimming_flag",
                                #'ACF_max_difference',     
                                'model_type'])

"""ts = int(file[-14:-4])
day_of_week = pd.to_datetime(ts,unit='s').dayofweek
ts = ts + ((3600 * 24) * (6-day_of_week))
ts = ts + ((24*3600) - ts % (24*3600))
end = pd.to_datetime(ts, unit='s')
last_week_start = end-pd.Timedelta('1 w')
start = end-pd.Timedelta('4 w')"""

In [4]:
col_id = "lte_eci" #! debug, ezt elvileg a címből kéne kiolvasni
df = data_step( file , col_id)
labels = df[col_id].unique()

In [16]:
end = df.dt.max()
start = end-pd.Timedelta('4 w')
last_week_start = end-pd.Timedelta('1 w')

In [5]:
import dask
from dask.distributed import Client
client = Client( dashboard_address = ':44594', n_workers = 68, threads_per_worker = 2 ) #scheduler_address=':37243'
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44594/status,
Dashboard: http://127.0.0.1:44594/status,Workers: 68
Total threads: 136,Total memory: 314.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43995,Workers: 68
Dashboard: http://127.0.0.1:44594/status,Total threads: 136
Started: Just now,Total memory: 314.49 GiB
Comm: tcp://127.0.0.1:45180,Total threads: 2
Dashboard: http://127.0.0.1:39929/status,Memory: 4.62 GiB
Nanny: tcp://127.0.0.1:40197,


In [6]:
df.ts.min()

1611734400

In [26]:
def preproc(df_label):
    df_label = clean(df_label, col_id)

    if (df_label.shape[0]<3): return pd.DataFrame()

    time_slice = pd.date_range(start, end, freq='H')
    df_label = df_label.drop_duplicates(subset = ["dt", col_id])
    df_label = df_label.set_index(["dt", col_id])

    new_multi_index = pd.MultiIndex.from_product([time_slice,
                                                    df_label.index.levels[1]],names = ["dt", col_id])
    df_label = df_label.reindex( new_multi_index )
    df_label = df_label.reset_index()
    return df_label

In [23]:
def one_dim(df2):
    if df2.empty: #df2 == None -> True
        return df2
    #df2 = df[[col_id, col, "dt"]]

    #df2 = df2.sort_values([col_id,"dt"])
    #*df2 = df2.drop_duplicates(subset=[col_id, "dt"], keep="last")
    #*df2 = df2.pivot(index=col_id, columns="dt", values = col)
    df2 = df2.melt(id_vars = [col_id, "dt" ], value_vars = df2.columns.drop([col_id, "ts", "dt"]))
    df2 = df2.pivot(index = ["variable",col_id], columns = "dt", values ="value")

    ts_cols_fun = lambda col: type(col) == pd._libs.tslibs.timestamps.Timestamp
    ts_cols = [*filter(ts_cols_fun, df2.columns)]

    df2["missing_data_ratio_all"] = df2.apply(lambda x: 1-sum(~x.isna())/(4*7*24), axis=1)
    #last_week_start_idx = len(ts_cols) - 168
    #df2["missing_data_ratio_last_week"] = df2.apply(lambda x: 1-sum(~x[last_week_start_idx:].isna())/(7*24),axis=1)
    df2["missing_data_ratio_last_week"] = df2.apply(lambda x: 1-sum(~x[last_week_start:].isna())/(7*24),axis=1)
    df2["path"] = file
    df2["ts"] = ts
    df2["table"] = col_id + str(ts)
    df2["dimension_name"] = df2.apply(lambda x: OrderedDict([(col_id, x.name[1])]), axis=1)
    df2["kpi_name"] = df2.apply(lambda x: x.name[0], axis = 1)
    df2["model_type"] = df2.apply(lambda x: "bad_quality_data" if x["missing_data_ratio_all"] > max_missing_ratio else None, axis=1)


    # x.name[0] - kpi name of ts row
    df2["model_type"] = df2.apply(lambda x: "constant" 
                        if ((x[:-6].quantile(0.51)==kpi_constraints_dict[ x.name[0] ]) or (x[:-6].quantile(0.49)==kpi_constraints_dict[ x.name[0] ]) or (x[:-6].std() < 10**(-14))) & (x.model_type!="bad_quality_data") 
                        else x.model_type, axis=1)
    #*upper, lower = kpi_constraints_dict[col]
    #*df2["model_type"] = df2.apply(lambda x: "constant" if ((x[:-6].quantile(0.51)==lower) or (x[:-6].quantile(0.49)==upper))&(x.model_type!="bad_quality_data") else x.model_type, axis=1)

    #model_type_lambda2 = lambda x: "constant" if ((x[:-6].quantile(0.51)==lower) or (x[:-6].quantile(0.49)==upper))&(x.model_type!="bad_quality_data") else x.model_type
    #df2["model_type"] = model_type_lambda2(df2)

    df_bad_quality = df2[(df2["model_type"]=="bad_quality_data")|(df2["model_type"]=="constant")]

    df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None

    df2 = df2[df2["model_type"].isna()]
    if df2.shape[0]>0:
        df2[["nan_trimming_flag", "new_start", "new_end"]] = df2.apply(lambda x: drop_missing_data(np.array(x[:-8].dropna(axis=0).index, dtype='datetime64[h]'), 4),axis=1, result_type='expand')
        
        df2["missing_data_drop_flag"] = df2["nan_trimming_flag"]
        df2["start_index"] = df2.apply(lambda x: df2.columns.get_loc(pd.Timestamp(x["new_start"])), axis=1)
        df2["end_index"] = df2.apply(lambda x: df2.columns.get_loc(pd.Timestamp(x["new_end"])), axis=1)
        
        
        df_change = df2[df2.nan_trimming_flag==True]
        df2 = df2[df2.nan_trimming_flag!=True]

        df_change[["missing_data_imputation_flag", "missing_data_drop_flag", "seasonality_flag"]] = df_change.apply(lambda x: (True, 1, test_seasonality_vectorized((pd.Series(x[x["start_index"]:x["end_index"]]).interpolate(method='pchip')))) 
                                                                            if sum(x[x["start_index"]:x["end_index"]].isna())!=0 
                                                                            else (False, 1, test_seasonality_vectorized(x[x["start_index"]:x["end_index"]])),
                                                                            axis=1, result_type='expand')

        # Ez lehet x[0:667]-re kene hogy fusson
        if df2.shape[0]>0:
            #df2[["missing_data_imputation_flag", "missing_data_drop_flag", "is_seas"]] = df2.apply(lambda x: (True, 0, test_seasonality_vectorized((pd.Series(x[x["start_index"]:x["end_index"]]).interpolate(method='pchip')))) if sum(x[x["start_index"]:x["end_index"]].isna())!=0 else (False, 0, test_seasonality_vectorized(x[x["start_index"]:x["end_index"]])),axis=1, result_type='expand')
            df2[["missing_data_imputation_flag", "missing_data_drop_flag", "seasonality_flag"]] = df2.apply(lambda x: (True, 0, test_seasonality_vectorized((pd.Series(x[0:667]).interpolate(method='pchip')))) 
                                                                                                                if sum(x[0:667].isna())!=0 
                                                                                                                else (False, 0, test_seasonality_vectorized(x[0:667])),axis=1, result_type='expand')
        
        df2 = pd.concat([df2, df_change])

        df_change = None

        df_trend = df2[df2.seasonality_flag==False]
        df2 = df2[df2.seasonality_flag!=False]
        if df_trend.shape[0]>0:
            ts_cols = [*filter(ts_cols_fun, df_trend.columns)]
            df_trend["statonarity_flag"] = df_trend.apply(lambda x: test_stationarity(x[ ts_cols ].dropna(),dir_dict[ x.name[0] ]) ,axis=1)
        else:
            df2["statonarity_flag"] = False #None
        df2 = pd.concat([df2, df_trend])

        df_trend = None
        df2.loc[(df2['seasonality_flag'] == True), 'model_type'] = 'seasonal_prophet'
        df2.loc[(df2['seasonality_flag'] != True) & (df2['statonarity_flag'] == True), 'model_type'] = 'non_seasonal_dbscan'
        df2.loc[(df2['seasonality_flag'] != True) & (df2['statonarity_flag'] != True), 'model_type'] = 'non_seasonal_trend'
    return df2[df2.columns[-17:]].reset_index(drop = True)

In [24]:
dim_vals = df[ col_id ].unique()

In [25]:
df_future = client.scatter(df)
_df = preproc(_dim_vals, df_future)
one_dim(_df)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


CommClosedError: in <TCP (closed) ConnectionPool.scatter local=tcp://127.0.0.1:49006 remote=tcp://127.0.0.1:43995>: Stream is closed

In [17]:
size = 40
tasks = []

for idx in range( dim_vals.shape[0]//size + 1 ):
    _dim_vals = dim_vals[idx * size: (idx+1) * size]
    df_label = df[ df[col_id].isin(_dim_vals) ]

    df_label_dask = client.scatter(df_label) 

    _df = dask.delayed(preproc)(df_label_dask)
    task = dask.delayed(one_dim)(_df)

    tasks.append(task)

res = dask.compute(tasks)[0]

results = pd.concat(res).reset_index(drop=True)
results.to_csv("/home/jovyan/work/elbaanh/AIO-dev/metadata_result.csv")

KeyError: Timestamp('2021-02-17 08:00:00')

In [27]:
results = pd.concat(res[:-1])
results.dimension_name = results.dimension_name.map(str)

(143196, 26)

In [28]:
results.to_csv("/home/jovyan/work/elbaanh/AIO-dev/metadata_result-lte_enodeb_id.csv")

distributed.utils - ERROR - '<' not supported between instances of 'NoneType' and 'NoneType'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/utils.py", line 680, in log_errors
    yield
  File "/opt/conda/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 3403, in graph_doc
    graph = TaskGraph(scheduler, sizing_mode="stretch_both")
  File "/opt/conda/lib/python3.8/site-packages/distributed/dashboard/components/scheduler.py", line 1940, in __init__
    self.layout = GraphLayout(scheduler)
  File "/opt/conda/lib/python3.8/site-packages/distributed/diagnostics/graph_layout.py", line 39, in __init__
    self.update_graph(
  File "/opt/conda/lib/python3.8/site-packages/distributed/diagnostics/graph_layout.py", line 49, in update_graph
    stack = sorted(tasks, key=lambda k: priority.get(k, 0), reverse=True)
TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'
tornado.application - ERROR - Un

In [16]:
pd.Series(list(map(type,res))) == type

0      False
1      False
2      False
3      False
4      False
       ...  
313    False
314    False
315    False
316    False
317     True
Length: 318, dtype: bool

In [ ]:
results = pd.concat(res).reset_index(drop=True)
results.to_csv("/home/jovyan/work/elbaanh/AIO-dev/metadata_result.csv")

In [16]:
df.shape[0]/600 * 84

1819523.8599999999

In [10]:
# 81600 - 5 min, size 3
# 

(81674, 17)

# Timing

In [8]:
import time


df_list = []
for size in [5, 10, 30, 100, 200]: # batch size

    for num_batches in [200, 400, 500]:
        try:
            tasks = []
            t1 = time.time()
            for idx in range(num_batches):     #number of batches
                _dim_vals = dim_vals[idx * size: (idx+1) * size]
                df_label = df[ df[col_id].isin(_dim_vals) ]

                
                df_label_dask = client.scatter(df_label)

                _df = dask.delayed(preproc)(df_label_dask)
                task = dask.delayed(one_dim)(_df)

                tasks.append(task)
            
            res = dask.compute(tasks)[0]
            results = pd.concat(res).reset_index(drop = True)
        except Exception as e:
            with open("/home/jovyan/work/elbaanh/AIO-dev/measure_md.txt", "a") as f:
                f.write(e)

        delta_t = time.time()-t1
        s = f"Size: {size} \t num batches:{num_batches} \t took {delta_t} s \t { delta_t / (len(results))} s/ts \t Num dims: {len(dim_vals)}\n"
        with open("/home/jovyan/work/elbaanh/AIO-dev/measure_md.txt", "a") as f:
            f.write(s)

KeyboardInterrupt: 

In [8]:
def calc_ties(x):
    args = list(map(np.asarray, x))
    alldata = np.concatenate(args)
    ranked = rankdata(alldata)
    ties = tiecorrect(ranked)
    
    return ties

In [9]:
def _prep_df(df2):
    df2 = df2.melt(id_vars = [col_id, "dt" ], value_vars = df2.columns.drop([col_id, "ts", "dt"]))
    df2 = df2.pivot(index = ["variable",col_id], columns = "dt", values ="value")
    df2["missing_data_ratio_all"] = df2.apply(lambda x: 1-sum(~x.isna())/(4*7*24), axis=1)
    df2["missing_data_ratio_last_week"] = df2.apply(lambda x: 1-sum(~x[last_week_start:].isna())/(7*24),axis=1)
    df2["path"] = file
    df2["ts"] = ts
    df2["table"] = col_id + str(ts)
    df2["dimension_name"] = df2.apply(lambda x: OrderedDict([(col_id, x.name[1])]), axis=1)
    df2["kpi_name"] = df2.apply(lambda x: x.name[0], axis = 1)
    df2["model_type"] = df2.apply(lambda x: "bad_quality_data" if x["missing_data_ratio_all"] > max_missing_ratio else None, axis=1)


    # x.name[0] - kpi name of ts row
    df2.apply(lambda x: "constant" 
                        if ((x[:-6].quantile(0.51)==kpi_constraints_dict[ x.name[0] ]) or (x[:-6].quantile(0.49)==kpi_constraints_dict[ x.name[0] ])) & (x.model_type!="bad_quality_data") 
                        else x.model_type, axis=1)
    #*upper, lower = kpi_constraints_dict[col]
    #*df2["model_type"] = df2.apply(lambda x: "constant" if ((x[:-6].quantile(0.51)==lower) or (x[:-6].quantile(0.49)==upper))&(x.model_type!="bad_quality_data") else x.model_type, axis=1)

    #model_type_lambda2 = lambda x: "constant" if ((x[:-6].quantile(0.51)==lower) or (x[:-6].quantile(0.49)==upper))&(x.model_type!="bad_quality_data") else x.model_type
    #df2["model_type"] = model_type_lambda2(df2)

    df_bad_quality = df2[(df2["model_type"]=="bad_quality_data")|(df2["model_type"]=="constant")]

    df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None

    df2 = df2[df2["model_type"].isna()]
    if df2.shape[0]>0:
        df2[["nan_trimming_flag", "new_start", "new_end"]] = df2.apply(lambda x: drop_missing_data(np.array(x[:-8].dropna(axis=0).index, dtype='datetime64[h]'), 4),axis=1, result_type='expand')
        
        df2["missing_data_drop_flag"] = df2["nan_trimming_flag"]
        df2["start_index"] = df2.apply(lambda x: df2.columns.get_loc(pd.Timestamp(x["new_start"])), axis=1)
        df2["end_index"] = df2.apply(lambda x: df2.columns.get_loc(pd.Timestamp(x["new_end"])), axis=1)
        
        
        df_change = df2[df2.nan_trimming_flag==True]
        df2 = df2[df2.nan_trimming_flag!=True]
    
    return df2

In [108]:
from scipy.stats import tiecorrect, rankdata


for df in df_list:
    df = preproc(df)
    df = _prep_df(df)
    ties = df.apply(calc_ties, axis = 1)
    print(ties)

<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be 

Empty DataFrame
Columns: [2021-01-27 08:00:00, 2021-01-27 09:00:00, 2021-01-27 10:00:00, 2021-01-27 11:00:00, 2021-01-27 12:00:00, 2021-01-27 13:00:00, 2021-01-27 14:00:00, 2021-01-27 15:00:00, 2021-01-27 16:00:00, 2021-01-27 17:00:00, 2021-01-27 18:00:00, 2021-01-27 19:00:00, 2021-01-27 20:00:00, 2021-01-27 21:00:00, 2021-01-27 22:00:00, 2021-01-27 23:00:00, 2021-01-28 00:00:00, 2021-01-28 01:00:00, 2021-01-28 02:00:00, 2021-01-28 03:00:00, 2021-01-28 04:00:00, 2021-01-28 05:00:00, 2021-01-28 06:00:00, 2021-01-28 07:00:00, 2021-01-28 08:00:00, 2021-01-28 09:00:00, 2021-01-28 10:00:00, 2021-01-28 11:00:00, 2021-01-28 12:00:00, 2021-01-28 13:00:00, 2021-01-28 14:00:00, 2021-01-28 15:00:00, 2021-01-28 16:00:00, 2021-01-28 17:00:00, 2021-01-28 18:00:00, 2021-01-28 19:00:00, 2021-01-28 20:00:00, 2021-01-28 21:00:00, 2021-01-28 22:00:00, 2021-01-28 23:00:00, 2021-01-29 00:00:00, 2021-01-29 01:00:00, 2021-01-29 02:00:00, 2021-01-29 03:00:00, 2021-01-29 04:00:00, 2021-01-29 05:00:00, 2021-01-

<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be 

Empty DataFrame
Columns: [2021-01-27 08:00:00, 2021-01-27 09:00:00, 2021-01-27 10:00:00, 2021-01-27 11:00:00, 2021-01-27 12:00:00, 2021-01-27 13:00:00, 2021-01-27 14:00:00, 2021-01-27 15:00:00, 2021-01-27 16:00:00, 2021-01-27 17:00:00, 2021-01-27 18:00:00, 2021-01-27 19:00:00, 2021-01-27 20:00:00, 2021-01-27 21:00:00, 2021-01-27 22:00:00, 2021-01-27 23:00:00, 2021-01-28 00:00:00, 2021-01-28 01:00:00, 2021-01-28 02:00:00, 2021-01-28 03:00:00, 2021-01-28 04:00:00, 2021-01-28 05:00:00, 2021-01-28 06:00:00, 2021-01-28 07:00:00, 2021-01-28 08:00:00, 2021-01-28 09:00:00, 2021-01-28 10:00:00, 2021-01-28 11:00:00, 2021-01-28 12:00:00, 2021-01-28 13:00:00, 2021-01-28 14:00:00, 2021-01-28 15:00:00, 2021-01-28 16:00:00, 2021-01-28 17:00:00, 2021-01-28 18:00:00, 2021-01-28 19:00:00, 2021-01-28 20:00:00, 2021-01-28 21:00:00, 2021-01-28 22:00:00, 2021-01-28 23:00:00, 2021-01-29 00:00:00, 2021-01-29 01:00:00, 2021-01-29 02:00:00, 2021-01-29 03:00:00, 2021-01-29 04:00:00, 2021-01-29 05:00:00, 2021-01-

<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be 

...

<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be 

Empty DataFrame
Columns: [2021-01-27 08:00:00, 2021-01-27 09:00:00, 2021-01-27 10:00:00, 2021-01-27 11:00:00, 2021-01-27 12:00:00, 2021-01-27 13:00:00, 2021-01-27 14:00:00, 2021-01-27 15:00:00, 2021-01-27 16:00:00, 2021-01-27 17:00:00, 2021-01-27 18:00:00, 2021-01-27 19:00:00, 2021-01-27 20:00:00, 2021-01-27 21:00:00, 2021-01-27 22:00:00, 2021-01-27 23:00:00, 2021-01-28 00:00:00, 2021-01-28 01:00:00, 2021-01-28 02:00:00, 2021-01-28 03:00:00, 2021-01-28 04:00:00, 2021-01-28 05:00:00, 2021-01-28 06:00:00, 2021-01-28 07:00:00, 2021-01-28 08:00:00, 2021-01-28 09:00:00, 2021-01-28 10:00:00, 2021-01-28 11:00:00, 2021-01-28 12:00:00, 2021-01-28 13:00:00, 2021-01-28 14:00:00, 2021-01-28 15:00:00, 2021-01-28 16:00:00, 2021-01-28 17:00:00, 2021-01-28 18:00:00, 2021-01-28 19:00:00, 2021-01-28 20:00:00, 2021-01-28 21:00:00, 2021-01-28 22:00:00, 2021-01-28 23:00:00, 2021-01-29 00:00:00, 2021-01-29 01:00:00, 2021-01-29 02:00:00, 2021-01-29 03:00:00, 2021-01-29 04:00:00, 2021-01-29 05:00:00, 2021-01-

<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-105-305ce4c46fea>:26: SettingWithCopyWarning: 
A value is trying to be 

Empty DataFrame
Columns: [2021-01-27 08:00:00, 2021-01-27 09:00:00, 2021-01-27 10:00:00, 2021-01-27 11:00:00, 2021-01-27 12:00:00, 2021-01-27 13:00:00, 2021-01-27 14:00:00, 2021-01-27 15:00:00, 2021-01-27 16:00:00, 2021-01-27 17:00:00, 2021-01-27 18:00:00, 2021-01-27 19:00:00, 2021-01-27 20:00:00, 2021-01-27 21:00:00, 2021-01-27 22:00:00, 2021-01-27 23:00:00, 2021-01-28 00:00:00, 2021-01-28 01:00:00, 2021-01-28 02:00:00, 2021-01-28 03:00:00, 2021-01-28 04:00:00, 2021-01-28 05:00:00, 2021-01-28 06:00:00, 2021-01-28 07:00:00, 2021-01-28 08:00:00, 2021-01-28 09:00:00, 2021-01-28 10:00:00, 2021-01-28 11:00:00, 2021-01-28 12:00:00, 2021-01-28 13:00:00, 2021-01-28 14:00:00, 2021-01-28 15:00:00, 2021-01-28 16:00:00, 2021-01-28 17:00:00, 2021-01-28 18:00:00, 2021-01-28 19:00:00, 2021-01-28 20:00:00, 2021-01-28 21:00:00, 2021-01-28 22:00:00, 2021-01-28 23:00:00, 2021-01-29 00:00:00, 2021-01-29 01:00:00, 2021-01-29 02:00:00, 2021-01-29 03:00:00, 2021-01-29 04:00:00, 2021-01-29 05:00:00, 2021-01-

KeyboardInterrupt: 

<ipython-input-90-b6d966c2d527>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-90-b6d966c2d527>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-90-b6d966c2d527>:31: SettingWithCopyWarning: 
A value is trying to be set

dt,missing_data_ratio_all,missing_data_ratio_last_week,path,ts,table,dimension_name,kpi_name,model_type,nan_trimming_flag,new_start,new_end,missing_data_drop_flag,start_index,end_index,missing_data_imputation_flag,seasonality_flag,statonarity_flag
0,0.275298,0.964286,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21778197.0},abnormal_lte_conn_rel_ratio_sample,seasonal_prophet,True,2021-01-27 18:00:00,2021-02-24 08:00:00,1,10,672,True,True,NaN
1,0.084821,0.696429,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21799446.0},abnormal_lte_conn_rel_ratio_sample,seasonal_prophet,True,2021-01-29 16:00:00,2021-02-24 08:00:00,1,56,672,True,True,NaN
2,0.014881,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 25355010.0},abnormal_lte_conn_rel_ratio_sample,seasonal_prophet,True,2021-01-28 22:00:00,2021-02-24 08:00:00,1,38,672,True,True,NaN
3,0.275298,0.964286,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21778197.0},abnormal_lte_conn_rel_ratio_sum,seasonal_prophet,True,2021-01-27 18:00:00,2021-02-24 08:00:00,1,10,672,True,True,NaN
4,0.084821,0.696429,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21799446.0},abnormal_lte_conn_rel_ratio_sum,seasonal_prophet,True,2021-01-29 16:00:00,2021-02-24 08:00:00,1,56,672,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.211310,0.773810,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21799446.0},rtt_net_wmean,non_seasonal_dbscan,True,2021-02-01 13:00:00,2021-02-24 05:00:00,1,125,669,True,False,True
147,0.010417,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 25355010.0},rtt_net_wmean,non_seasonal_trend,True,2021-01-28 10:00:00,2021-02-24 08:00:00,1,26,672,True,False,False
148,0.147321,0.755952,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21799446.0},tcp_tp_dl_wmean,non_seasonal_dbscan,True,2021-01-31 10:00:00,2021-02-24 08:00:00,1,98,672,True,False,True
149,0.016369,0.696429,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 25355010.0},tcp_tp_ul_wmean,non_seasonal_trend,True,2021-01-28 16:00:00,2021-02-24 08:00:00,1,32,672,True,False,False


<ipython-input-37-52e8884a34bd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-37-52e8884a34bd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-37-52e8884a34bd>:31: SettingWithCopyWarning: 
A value is trying to be set

dt,missing_data_ratio_all,missing_data_ratio_last_week,path,ts,table,dimension_name,kpi_name,model_type,nan_trimming_flag,new_start,new_end,missing_data_drop_flag,start_index,end_index,missing_data_imputation_flag,seasonality_flag,statonarity_flag
0,0.053571,0.863095,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21658645.0},bytes_dl_sum,seasonal_prophet,False,2021-01-27 08:00:00,2021-02-22 21:00:00,0,0,637,True,True,NaN
1,0.058036,0.869048,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21658645.0},serving_rsrp_sample,seasonal_prophet,False,2021-01-27 08:00:00,2021-02-22 20:00:00,0,0,636,True,True,NaN
2,0.058036,0.869048,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21658645.0},serving_rsrp_sum,seasonal_prophet,False,2021-01-27 08:00:00,2021-02-22 20:00:00,0,0,636,True,True,NaN
3,0.058036,0.869048,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21658645.0},serving_rsrq_sample,seasonal_prophet,False,2021-01-27 08:00:00,2021-02-22 20:00:00,0,0,636,True,True,NaN
4,0.058036,0.869048,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21658645.0},serving_rsrq_sum,seasonal_prophet,False,2021-01-27 08:00:00,2021-02-22 20:00:00,0,0,636,True,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,0.136905,0.791667,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21761037.0},x2_ho_wmean,non_seasonal_dbscan,True,2021-01-31 06:00:00,2021-02-24 08:00:00,1,94,672,True,False,True
943,0.151786,0.815476,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 21770557.0},x2_ho_wmean,non_seasonal_trend,True,2021-01-31 09:00:00,2021-02-24 08:00:00,1,97,672,True,False,False
944,0.010417,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 23854869.0},x2_ho_wmean,non_seasonal_trend,True,2021-01-28 10:00:00,2021-02-24 08:00:00,1,26,672,True,False,False
945,0.066964,0.702381,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,lte_eci1614556800,{'lte_eci': 25477005.0},x2_ho_wmean,non_seasonal_trend,True,2021-01-30 05:00:00,2021-02-24 07:00:00,1,69,671,True,False,False


ValueError: All numbers are identical in kruskal

In [ ]:
_dim_vals.shape

In [54]:
df_list = []


size = 300
d = len(dim_vals) // size 




for idx in range(d):
    _dim_vals = dim_vals[ idx * size: (idx+1) * size ]  
    _df = df[ df[col_id].isin(_dim_vals) ] 
    df_list.append(_df)

KeyboardInterrupt: 

32 | 31 s <br>
64

In [17]:
x = preproc(dim_vals[:5])

In [38]:
res = one_dim(x)

<ipython-input-37-52e8884a34bd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-37-52e8884a34bd>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-37-52e8884a34bd>:31: SettingWithCopyWarning: 
A value is trying to be set

In [21]:
df2 = x.melt(id_vars = [col_id, "dt" ], value_vars = df.columns.drop([col_id, "ts", "dt"]))
df2 = df2.pivot(index = ["variable",col_id], columns = "dt", values ="value")
df2["missing_data_ratio_all"] = df2.apply(lambda x: 1-sum(~x.isna())/(4*7*24), axis=1)
df2["missing_data_ratio_last_week"] = df2.apply(lambda x: 1-sum(~x[last_week_start:].isna())/(7*24),axis=1)
df2["path"] = file
df2["ts"] = ts
df2["table"] = col_id + str(ts)
df2["dimension_name"] = df2.apply(lambda x: OrderedDict([(col_id, x.name[1])]), axis=1)
df2["kpi"] = df2.apply(lambda x: x.name[0], axis = 1)

In [30]:
df2 = x.melt(id_vars = [col_id, "dt" ], value_vars = df.columns.drop([col_id, "ts", "dt"]))
df2 = df2.pivot(index = ["variable",col_id], columns = "dt", values ="value")
df2["missing_data_ratio_all"] = df2.apply(lambda x: 1-sum(~x.isna())/(4*7*24), axis=1)
df2["missing_data_ratio_last_week"] = df2.apply(lambda x: 1-sum(~x[last_week_start:].isna())/(7*24),axis=1)
df2["path"] = file
df2["ts"] = ts
df2["table"] = col_id + str(ts)
df2["dimension_name"] = df2.apply(lambda x: OrderedDict([(col_id, x.name)]), axis=1)
#df2["kpi_name"] = col
df2["model_type"] = df2.apply(lambda x: "bad_quality_data" if x["missing_data_ratio_all"] > max_missing_ratio else None, axis=1)


# x.name[0] - kpi name of ts row
df2.apply(lambda x: "constant" 
                    if ((x[:-6].quantile(0.51)==kpi_constraints_dict[ x.name[0] ]) or (x[:-6].quantile(0.49)==kpi_constraints_dict[ x.name[0] ])) & (x.model_type!="bad_quality_data") 
                    else x.model_type, axis=1)
#upper, lower = kpi_constraints_dict[col]
#*df2["model_type"] = df2.apply(lambda x: "constant" if ((x[:-6].quantile(0.51)==lower) or (x[:-6].quantile(0.49)==upper))&(x.model_type!="bad_quality_data") else x.model_type, axis=1)

#model_type_lambda2 = lambda x: "constant" if ((x[:-6].quantile(0.51)==lower) or (x[:-6].quantile(0.49)==upper))&(x.model_type!="bad_quality_data") else x.model_type
#df2["model_type"] = model_type_lambda2(df2)

df_bad_quality = df2[(df2["model_type"]=="bad_quality_data")|(df2["model_type"]=="constant")]

df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None

df2 = df2[df2["model_type"].isna()]
if df2.shape[0]>0:
    df2[["nan_trimming_flag", "new_start", "new_end"]] = df2.apply(lambda x: drop_missing_data(np.array(x[:-8].dropna(axis=0).index, dtype='datetime64[h]'), 4),axis=1, result_type='expand')
    
    df2["missing_data_drop_flag"] = df2["nan_trimming_flag"]
    df2["start_index"] = df2.apply(lambda x: df2.columns.get_loc(pd.Timestamp(x["new_start"])), axis=1)
    df2["end_index"] = df2.apply(lambda x: df2.columns.get_loc(pd.Timestamp(x["new_end"])), axis=1)
    
    
    df_change = df2[df2.nan_trimming_flag==True]
    df2 = df2[df2.nan_trimming_flag!=True]

    df_change[["missing_data_imputation_flag", "missing_data_drop_flag", "seasonality_flag"]] = df_change.apply(lambda x: (True, 1, test_seasonality_vectorized((pd.Series(x[x["start_index"]:x["end_index"]]).interpolate(method='pchip')))) 
                                                                        if sum(x[x["start_index"]:x["end_index"]].isna())!=0 
                                                                        else (False, 1, test_seasonality_vectorized(x[x["start_index"]:x["end_index"]])),
                                                                        axis=1, result_type='expand')

    # Ez lehet x[0:667]-re kene hogy fusson
    if df2.shape[0]>0:
        #df2[["missing_data_imputation_flag", "missing_data_drop_flag", "is_seas"]] = df2.apply(lambda x: (True, 0, test_seasonality_vectorized((pd.Series(x[x["start_index"]:x["end_index"]]).interpolate(method='pchip')))) if sum(x[x["start_index"]:x["end_index"]].isna())!=0 else (False, 0, test_seasonality_vectorized(x[x["start_index"]:x["end_index"]])),axis=1, result_type='expand')
        df2[["missing_data_imputation_flag", "missing_data_drop_flag", "seasonality_flag"]] = df2.apply(lambda x: (True, 0, test_seasonality_vectorized((pd.Series(x[0:667]).interpolate(method='pchip')))) 
                                                                                                            if sum(x[0:667].isna())!=0 
                                                                                                            else (False, 0, test_seasonality_vectorized(x[0:667])),axis=1, result_type='expand')
    
    df2 = pd.concat([df2, df_change])

    df_change = None

    df_trend = df2[df2.seasonality_flag==False]
    df2 = df2[df2.seasonality_flag!=False]

<ipython-input-30-0148fc2d432e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-30-0148fc2d432e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad_quality[['seasonality_flag','statonarity_flag','missing_data_imputation_flag', "nan_trimming_flag"]] = None,None,None,None
<ipython-input-30-0148fc2d432e>:25: SettingWithCopyWarning: 
A value is trying to be set

In [72]:
del df_list

In [71]:
compute = dask.compute(df_list)[0]

KeyboardInterrupt: 

In [39]:
col_name = df[ col_id ].unique()[7]

tasks = []
for col_name in df[ col_id ].unique():
    _df = df[ df[col_id] == col_name ]
    df2 = dask.delayed(one_kpi)(_df)
    tasks.append(df2)

#df[ df[col_id] == col_name ].melt(id_vars = [col_id, "dt" ], value_vars = df.columns.drop([col_id, "ts", "dt"])).pivot(index = ["variable",col_id], columns = "dt", values ="value")

In [41]:
one_kpi(_df)

KeyError: Timestamp('2021-02-22 00:00:00')

In [40]:
results = dask.compute(df2)[0]

KeyError: Timestamp('2021-02-22 00:00:00')

In [49]:
df[[col_id, kpi, "dt"]]

,plan_name,ims_sess_setup_succ_ratio_wmean,dt
311275,mPERS Test Plan,NaN,2021-01-27 08:00:00
198875,Simply Prepaid 10 GB,1.000000,2021-01-27 08:00:00
161173,Uncarrier 10 GB,0.980392,2021-01-27 08:00:00
132667,Uncarrier 2.5 GB,0.926230,2021-01-27 08:00:00
129586,Uncarrier 7 GB,NaN,2021-01-27 08:00:00
...,...,...,...
131842,Gov HS First Responder UNL,NaN,2021-02-24 08:00:00
75882,Uncarrier 8.5 GB,NaN,2021-02-24 08:00:00
148593,Uncarrier 20.5 GB,NaN,2021-02-24 08:00:00
250896,One UnLimited,0.983466,2021-02-24 08:00:00


In [31]:
for col_name in df[ col_id ].unique():
    df[ df[col_id] == col_name ]

In [11]:
tasks = []
for kpi in df.columns.drop([col_id, "ts", "dt"]):
    if col not in dir_dict:
        continue
    _df = df[[col_id, col, "dt"]])
    df2 = dask.delayed(one_kpi)(_df, col)
    tasks.append(df2)

In [12]:
results = dask.compute(tasks)[0] # 40 s

/opt/conda/lib/python3.8/site-packages/distributed/worker.py:4423: UserWarning: Large object of size 2.15 MiB detected in task graph: 
  (                         plan_name  ims_sess_setu ... c_ratio_wmean')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


In [ ]:
pd.concat(results, ignore_index = True)

dt,missing_data_ratio_all,missing_data_ratio_last_week,path,ts,table,dimension_name,kpi_name,model_type,nan_trimming_flag,new_start,new_end,missing_data_drop_flag,start_index,end_index,missing_data_imputation_flag,seasonality_flag,statonarity_flag
0,0.034226,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '10GB 4G LTE (0 HS included)'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 06:00:00,2021-02-24 08:00:00,1,22,665,True,True,False
1,0.031250,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '2018 One Plus UnLimited 20GB Te...,rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 08:00:00,2021-02-24 08:00:00,1,24,665,True,True,False
2,0.034226,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '30 MB'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 06:00:00,2021-02-24 08:00:00,1,22,665,True,True,False
3,0.181548,0.726190,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '5 GB'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-29 14:00:00,2021-02-24 07:00:00,1,54,664,True,True,False
4,0.040179,0.690476,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': 'Ares 10GB'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 12:00:00,2021-02-24 08:00:00,1,28,665,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,0.032738,0.690476,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': 'Uncarrier 2.5 GB'},ims_sess_setup_succ_ratio_wmean,seasonal_prophet,True,2021-01-28 06:00:00,2021-02-24 08:00:00,1,22,665,True,True,NaN
1814,0.034226,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': 'Uncarrier 500 MB'},ims_sess_setup_succ_ratio_wmean,seasonal_prophet,True,2021-01-28 06:00:00,2021-02-24 08:00:00,1,22,665,True,True,NaN
1815,0.031250,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': 'Unlimited'},ims_sess_setup_succ_ratio_wmean,seasonal_prophet,True,2021-01-28 07:00:00,2021-02-24 08:00:00,1,23,665,True,True,NaN
1816,0.032738,0.684524,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': 'MPCS 500 MB'},ims_sess_setup_succ_ratio_wmean,non_seasonal_dbscan,True,2021-01-28 07:00:00,2021-02-24 08:00:00,1,23,665,True,False,True


In [57]:
results[2][0].head()

dt,2021-01-27 08:00:00,2021-01-27 09:00:00,2021-01-27 10:00:00,2021-01-27 11:00:00,2021-01-27 12:00:00,2021-01-27 13:00:00,2021-01-27 14:00:00,2021-01-27 15:00:00,2021-01-27 16:00:00,2021-01-27 17:00:00,2021-01-27 18:00:00,2021-01-27 19:00:00,2021-01-27 20:00:00,2021-01-27 21:00:00,2021-01-27 22:00:00,...,path,ts,table,dimension_name,kpi_name,model_type,nan_trimming_flag,new_start,new_end,missing_data_drop_flag,start_index,end_index,missing_data_imputation_flag,seasonality_flag,statonarity_flag
plan_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10GB 4G LTE (0 HS included),5.358440,1.760991,6.834164,2.417525,1.249019,2.108300,6.421797,12.244900,20.937958,47.975794,89.129137,97.263861,153.813297,50.763081,101.157589,...,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '10GB 4G LTE (0 HS included)'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 06:00:00,2021-02-24 08:00:00,1,22,665,True,True,False
2018 One Plus UnLimited 20GB Tether,21.948828,13.008780,8.239675,4.458641,11.271340,19.758229,59.896664,57.475048,208.230461,569.230488,1039.866782,1115.892487,1299.029944,763.878131,1213.686558,...,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '2018 One Plus UnLimited 20GB Te...,rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 08:00:00,2021-02-24 08:00:00,1,24,665,True,True,False
30 MB,0.321759,0.002279,0.516893,0.089523,0.000621,0.334113,0.661460,2.524805,6.290171,11.239630,29.876308,30.877732,53.066078,44.806950,61.982199,...,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '30 MB'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 06:00:00,2021-02-24 08:00:00,1,22,665,True,True,False
5 GB,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.034465,8.278836,1.781626,2.820418,4.372565,1.493806,1.028643,2.783531,...,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': '5 GB'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-29 14:00:00,2021-02-24 07:00:00,1,54,664,True,True,False
Ares 10GB,0.073374,0.033843,0.000000,0.000521,0.000326,0.175199,0.410192,0.119501,2.096086,9.906707,10.784743,22.205147,37.018923,15.081726,9.606184,...,/home/jovyan/work/elbaanh/AIO-dev/data/4weeks-...,1614556800,plan_name1614556800,{'plan_name': 'Ares 10GB'},rtp_packet_loss_dl_sum,seasonal_prophet,True,2021-01-28 12:00:00,2021-02-24 08:00:00,1,28,665,True,True,False


In [27]:
results_without_none = [*filter( None, results ) ]

In [35]:
_DF = DF.copy()
_DF["dimension_name"] = _DF["dimension_name"].map(str)
_DF[ ["dimension_name","kpi_name" ] ].drop_duplicates()

dt,dimension_name,kpi_name
0,"OrderedDict([('plan_name', '1 GB')])",bytes_dl_sum
1,"OrderedDict([('plan_name', '100 GB')])",bytes_dl_sum
2,"OrderedDict([('plan_name', '10GB 4G LTE (0 HS ...",bytes_dl_sum
3,"OrderedDict([('plan_name', '2 GB')])",bytes_dl_sum
4,"OrderedDict([('plan_name', '200 MB Throttle')])",bytes_dl_sum
...,...,...
30,"OrderedDict([('plan_name', 'Uncarrier 2.5 GB')])",ims_sess_setup_succ_ratio_wmean
31,"OrderedDict([('plan_name', 'Uncarrier 500 MB')])",ims_sess_setup_succ_ratio_wmean
32,"OrderedDict([('plan_name', 'Unlimited')])",ims_sess_setup_succ_ratio_wmean
33,"OrderedDict([('plan_name', 'MPCS 500 MB')])",ims_sess_setup_succ_ratio_wmean


In [1]:
pd.set_option('display.max_columns', 30)

NameError: name 'pd' is not defined

In [45]:
partial_df = pd.concat(results[0]).copy()
partial_df.dimension_name = partial_df.dimension_name.map(str)
partial_df.shape

(530, 17)

lol


In [28]:
DF = pd.concat([ pd.concat(res) for res in results_without_none ])

In [1]:
pd.concat([res for res in results])

NameError: name 'pd' is not defined

In [52]:
metadata_store = pd.read_csv("/home/jovyan/work/elbaanh/AIO-dev/data/metadata_store_1614556800.csv")
metadata_store.columns.sort_values()

<ipython-input-52-26a1593f343c>:1: DtypeWarning: Columns (2,9) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_store = pd.read_csv("/home/jovyan/work/elbaanh/AIO-dev/data/metadata_store_1614556800.csv")


Index(['ACF_max_difference', 'dimension_name', 'is_it_constant', 'kpi_name',
       'missing_data_imputation_flag', 'missing_data_ratio_all',
       'missing_data_ratio_last_week', 'model_type', 'nan_trimming_flag',
       'path', 'seasonality_flag', 'statonarity_flag', 'table', 'ts'],
      dtype='object')

In [60]:
metadata_store

,ts,path,dimension_name,kpi_name,missing_data_ratio_all,missing_data_ratio_last_week,seasonality_flag,statonarity_flag,missing_data_imputation_flag,table,nan_trimming_flag,ACF_max_difference,is_it_constant,model_type
0,1614556800,4weeks-subs_mcc-1614153600.csv,272,bytes_dl_sum,0.748512,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
1,1614556800,4weeks-subs_mcc-1614153600.csv,272,tcp_tp_ul_sum,0.980655,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
2,1614556800,4weeks-subs_mcc-1614153600.csv,272,tcp_tp_dl_sum,0.967262,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
3,1614556800,4weeks-subs_mcc-1614153600.csv,272,rtt_net_sum,0.967262,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
4,1614556800,4weeks-subs_mcc-1614153600.csv,272,tcp_tp_ul_wmean,0.980655,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131597,1614556800,4weeks-lte_eci-1614153600.csv,256088009.0,ims_call_setup_time_wmean,0.995536,0.982143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
1131598,1614556800,4weeks-lte_eci-1614153600.csv,256088009.0,ims_call_setup_delay_wmean,0.995536,0.982143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
1131599,1614556800,4weeks-lte_eci-1614153600.csv,256088009.0,rtp_delay_ul_wmean,0.995536,0.982143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data
1131600,1614556800,4weeks-lte_eci-1614153600.csv,256088009.0,rtp_delay_dl_wmean,0.995536,0.982143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bad_quality_data


In [24]:
col_num = 0
for label in labels:
    df_label = clean(df[df[col_id]==label], col_id=col_id)

    if (df_label.shape[0]<3): continue

    # reindexing, beacause timestamp may be missing
    time_slice = pd.date_range(df_label.dt.min(), df_label.dt.max(), freq='H')

    df_label = df_label.set_index('dt')
    df_label = df_label[~df_label.index.duplicated()].reindex(time_slice)
    df_list = [] 
    for col in df_label.columns.drop([col_id,"ts"]): #* ~ 10
        col_num +=1

In [25]:
col_num # 83 timeseries

4421

In [29]:
(9*60 + 23) / 4421 * 10**5 /60

212.24459021337557